In [77]:
ALL_ACTIONS = ("U","D","R","L")
class Grid:
    def __init__(self,dimension,start) -> None:
        rows , cols = dimension
        i , j = start
        self.cols = cols
        self.rows = rows
        self.i = i
        self.j = j
    def set(self,rewards,actions):
        self.rewards = rewards
        self.actions = actions
    def set_state(self,point):
        self.i = point[0]
        self.j = point[1]
    def get_current_state(self):
        return (self.i,self.j)
    def is_end(self,state):
        return state not in self.actions
    def get_next_state(self,state,action):
        (i,j) = state
        if action == 'U':
                i -= 1
        elif action == 'D':
            i += 1
        elif action == 'R':
            j += 1
        elif action == 'L':
            j -= 1
        return (i,j)
    def get_all_states(self):
        return set(self.actions.keys()) | set(self.rewards.keys())
    
    

In [78]:
def standard_grid():

    # .  .  .  1
    # .  x  . -1
    # s  .  .  .
    grid = Grid((3, 4), (2, 0))
    rewards = {(0, 3): 1, (1, 3): -1}
    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('L', 'R', 'U'),
        (2, 3): ('L', 'U'),
        }
    grid.set(rewards,actions)
    return grid

In [133]:
grid  = standard_grid()
transition_probs = {}
rewards = {}
for row in range(grid.rows):
    for col in range(grid.cols):
        state = (row,col)
        
        
        if not grid.is_end(state):
            for action in ALL_ACTIONS:
                next_state = grid.get_next_state(state,action)
                transition_probs[(state,action,next_state)] = 1
                if next_state in grid.rewards:
                    rewards[(state,action,next_state)] = grid.rewards[next_state]

print(rewards)
print("---------------")
print(transition_probs)

{((0, 2), 'R', (0, 3)): 1, ((1, 2), 'R', (1, 3)): -1, ((2, 3), 'U', (1, 3)): -1}
---------------
{((0, 0), 'U', (-1, 0)): 1, ((0, 0), 'D', (1, 0)): 1, ((0, 0), 'R', (0, 1)): 1, ((0, 0), 'L', (0, -1)): 1, ((0, 1), 'U', (-1, 1)): 1, ((0, 1), 'D', (1, 1)): 1, ((0, 1), 'R', (0, 2)): 1, ((0, 1), 'L', (0, 0)): 1, ((0, 2), 'U', (-1, 2)): 1, ((0, 2), 'D', (1, 2)): 1, ((0, 2), 'R', (0, 3)): 1, ((0, 2), 'L', (0, 1)): 1, ((1, 0), 'U', (0, 0)): 1, ((1, 0), 'D', (2, 0)): 1, ((1, 0), 'R', (1, 1)): 1, ((1, 0), 'L', (1, -1)): 1, ((1, 2), 'U', (0, 2)): 1, ((1, 2), 'D', (2, 2)): 1, ((1, 2), 'R', (1, 3)): 1, ((1, 2), 'L', (1, 1)): 1, ((2, 0), 'U', (1, 0)): 1, ((2, 0), 'D', (3, 0)): 1, ((2, 0), 'R', (2, 1)): 1, ((2, 0), 'L', (2, -1)): 1, ((2, 1), 'U', (1, 1)): 1, ((2, 1), 'D', (3, 1)): 1, ((2, 1), 'R', (2, 2)): 1, ((2, 1), 'L', (2, 0)): 1, ((2, 2), 'U', (1, 2)): 1, ((2, 2), 'D', (3, 2)): 1, ((2, 2), 'R', (2, 3)): 1, ((2, 2), 'L', (2, 1)): 1, ((2, 3), 'U', (1, 3)): 1, ((2, 3), 'D', (3, 3)): 1, ((2, 3), 'R'

In [46]:
def print_values(V, g):
    for i in range(g.rows):
        print("---------------------------")
        for j in range(g.cols):
            v = V.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")  
        print("")


def print_policy(P, g):
    for i in range(g.rows):
        print("---------------------------")
        for j in range(g.cols):
            a = P.get((i, j), " ")
            print("  %s  |" % a, end="")
        print("")
V={}
print_values(V,grid)

---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|


In [104]:
iteration = 0
policy = {
(2, 0): 'U',
(1, 0): 'U',
(0, 0): 'R',
(0, 1): 'R',
(0, 2): 'R',
(1, 2): 'U',
(2, 1): 'R',
(2, 2): 'U',
(2, 3): 'L',
(1, 1): 'X',
(1, 3): 'H',
(0, 3): 'G'
}
V= {}
for s in grid.get_all_states():
    V[s] = 0
while True:
    for state in grid.get_all_states():
        if not grid.is_end(state):
            new_v = 0
            for action in ALL_ACTIONS:
                for next_state in grid.get_all_states():
                    #deteministic actions:
                    action_prob = 1 if policy.get(state) == action else 0
                
                    r = rewards.get((state, action, next_state), 0)
                    new_v += action_prob * transition_probs.get((state, action, next_state), 0) * (r + 0.9 * V[next_state])
                    
                    
            V[state] = new_v
            
    
    
    iteration = iteration+1

    if iteration == 10:
        break
print_values(V, grid)

---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.90| 0.00|
---------------------------
 0.66| 0.73| 0.81| 0.73|


In [85]:
iteration = 0
policy = {
(2, 0): 'U',
(1, 0): 'U',
(0, 0): 'R',
(0, 1): 'R',
(0, 2): 'R',
(1, 2): 'U',
(2, 1): 'R',
(2, 2): 'U',
(2, 3): 'L',
(1, 1): 'X',
(1, 3): 'H',
(0, 3): 'G'
}

while True:
    is_policy_converged = True
    iteration += 1
    for state in grid.actions.keys():
        best_val = float('-inf')
        for action in ALL_ACTIONS:
            v = 0
            for next_state in grid.get_all_states():
                r = rewards.get((state,action,next_state),0)
                v += transition_probs.get((state,action,next_state),0) * (r + 0.9 *V[next_state])
            if v > best_val:
                best_action = action
                best_val = v
        if best_action != policy[state]:
            is_policy_converged = False
        policy[state] = best_action
    if is_policy_converged:
        break

print_policy(policy,grid)

---------------------------
  R  |  R  |  R  |  G  |
---------------------------
  U  |  X  |  U  |  H  |
---------------------------
  U  |  R  |  U  |  L  |


In [178]:
import numpy as np
q_table = np.zeros((grid.rows,grid.cols,len(ALL_ACTIONS)))
V= {}
for s in grid.get_all_states():
    V[s] = 0
iteration =0
policy = {
(2, 0): 'U',
(1, 0): 'U',
(0, 0): 'R',
(0, 1): 'R',
(0, 2): 'R',
(1, 2): 'U',
(2, 1): 'R',
(2, 2): 'U',
(2, 3): 'L',
(1, 1): 'X',
(1, 3): 'H',
(0, 3): 'G'
}
while True:
    biggest_change = 0  # To check for convergence
    for state in grid.get_all_states():
        if not grid.is_end(state):
            old_v = V[state]
            
            # Loop through each possible action to get the Q-values
            for action in ALL_ACTIONS:
                new_v = 0
                for next_state in grid.get_all_states():
                    # deterministic actions:
                    action_prob = 1 if policy.get(state) == action else 0
                    r = rewards.get((state, action, next_state), 0)
                    new_v += action_prob * transition_probs.get((state, action, next_state), 0) * (r + 0.9 * V[next_state])
                q_table[state[0], state[1], ALL_ACTIONS.index(action)] = new_v
                
            # Update V for the state with the max Q-value
            V[state] = np.max(q_table[state[0], state[1], :])
            
            # Check for the biggest change in V values
            biggest_change = max(biggest_change, abs(old_v - V[state]))
    iteration += 1
    # Stopping criteria: convergence or a fixed number of iterations
    if biggest_change < 1e-3 or iteration == 1000:
        break

# Extract policy based on Q-values
for state in grid.get_all_states():
    if not grid.is_end(state):
        policy[state] = ALL_ACTIONS[np.argmax(q_table[state[0], state[1], :])]



In [179]:
print_policy(policy,grid)

---------------------------
  R  |  R  |  R  |  G  |
---------------------------
  U  |  X  |  U  |  H  |
---------------------------
  U  |  R  |  U  |  L  |


In [180]:
iteration

4